### Problem setup

In [1]:
import time
import random
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
defects = pd.read_csv('defects.csv')
defects = defects.dropna()
defects = defects.drop_duplicates()
defects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       500 non-null    float64
 1   class   500 non-null    object 
dtypes: float64(1), object(1)
memory usage: 7.9+ KB


In [3]:
#on trie les défauts par position
defects = defects.sort_values(by=['x'])
defects['x'] = defects['x'].astype(int)
defects.head(20)

,x,class
479,0,a
141,0,a
142,1,a
493,2,b
125,2,a
403,3,a
61,3,b
72,8,c
112,9,c
33,9,c


In [4]:
import re

def display_defects():
    defects_count = defects.groupby(['x', 'class']).size().unstack(fill_value=0)
    defects_count = defects_count.reindex(range(500), fill_value=0)
    defects_count = defects_count.replace(0, '-')
    defects_count = defects_count.transpose()

    for i in range(3):
        line = defects_count.iloc[i].to_string()

        # Use regular expression to match and replace the pattern
        output_string = re.sub(r'(.)(?:\n\d+)', r'\1', line)
        res = output_string[1:].replace(" ", "")

        if i == 0:
            print("Class A")
        elif i == 1:
            print("Class B")
        else:
            print("Class C")
            
        for i in range(0, len(res), 100):
            print(res[i:i+100], end="\n")
        print("_"*100+"\n", end="\n")

display_defects()

Class A
2111--------1-1-1-----12-11--1--1---1-1-----1-1-1--1-1-1--11---------1---1--12-----1-2------12-1----
---1---11---1---1221----13--------------1--21---------1-1-11---3--1--1-11---------------311---------
---1-2------1-1-1-1----1---1--1-----------3---1----1-1----------------1----211-12---1---11----1-----
-11-1--------12--------------11-1-1-----1-------1-------1-----1-11-121-1-1------1----1-----11-------
-----11-11----1-1---------2--111-----12-1--1-12--1---1--11----1-1----1-----1-12--1112---11------11--
____________________________________________________________________________________________________

Class B
--11------1--------1-11-------1--11111------1-1-1--1--1----1--------1---2---------21------------1---
-------1--------11----11---1-1---------1-1----1-1----11-13---1-1--3-2--1-1--1--2----3111--111-------
1--------1---11-1----1-----1------11-----31--1--1--1-----111-------1-11-1--2-----1--11---1-1112-1---
-11-1-----1----1-----1--1-------2-11--1-----1--1---111-1-11-1--1-------1--

In [5]:
class Biscuit:

    def __init__(self, length, value, defects, id=None):
        self.length = length
        self.value = value
        self.defects = defects
        self.id = id

biscuit_0 = Biscuit(length=4, value=6, defects={"a":4, "b":2, "c":3}, id=0)
biscuit_1 = Biscuit(length=8, value=12, defects={"a":5, "b":4, "c":4}, id=1)
biscuit_2 = Biscuit(length=2, value=1, defects={"a":1, "b":2, "c":1}, id=2)
biscuit_3 = Biscuit(length=5, value=8, defects={"a":2, "b":3, "c":2}, id=3)

In [133]:
class Roll(Biscuit):
    
    def __init__(self, biscuits_and_positions, length):
        self.biscuits_and_positions = biscuits_and_positions
        self.defects = self.get_defects()
        self.length = length 
        self.score = self.get_score()
    
    def get_defects(self):
        defects_roll = {}
        for i in range(len(defects)):
            defects_roll[f"{defects.iloc[i]['x']}"] = defects.iloc[i]['class']
        return defects_roll
    
    def get_score(self):
        score = 0
        for biscuit in self.biscuits_and_positions:
            score += biscuit[0].value
        return score
    
    def can_be_added(self, biscuit, index, print_enabled=False):
        res = False
        biscuit = (biscuit, index)
        

        #on vérifie que le biscuit est bien dans le roll
        if biscuit[1]>=0 and biscuit[1]+biscuit[0].length-1<=self.length-1:
            res = True

            #on vérifie que le biscuit n'a pas trop de defauts
            defects_count = defects.groupby(['x', 'class']).size().unstack(fill_value=0)
            defects_count = defects_count.reindex(range(500), fill_value=0)
            defects_count = defects_count.transpose()

            start_index = biscuit[1]
            end_index = biscuit[1]+biscuit[0].length-1

            selected_range = defects_count.loc[:, start_index:end_index]
            count_defect_a = selected_range.loc['a'].sum()
            count_defect_b = selected_range.loc['b'].sum()
            count_defect_c = selected_range.loc['c'].sum()

            if count_defect_a > biscuit[0].defects['a']:
                res = False
                if print_enabled:
                    print("Too many defect a")
            elif count_defect_b > biscuit[0].defects['b']:
                res = False
                if print_enabled:
                    print("Too many defect b")
            elif count_defect_c > biscuit[0].defects['c']:
                res = False
                if print_enabled:
                    print("Too many defect c")

            #on vérifie que le biscuit ne chevauche pas un autre biscuit
            for biscuit_2 in self.biscuits_and_positions:
                start_1, end_1 = biscuit[1], biscuit[1] + biscuit[0].length -1
                start_2, end_2 = biscuit_2[1], biscuit_2[1] + biscuit_2[0].length -1

                if start_1 <= end_2 and end_1 >= start_2:
                    res = False
                    if print_enabled:
                        print("Biscuit overlap")
                    break
        if res and print_enabled:
            print("Biscuit can be added")
        return res
    
    def add_biscuit(self, biscuit, index, print_enabled=False):
        if print_enabled:
            print(f"Trying to add (biscuit_{biscuit.id}, {index}):")
        if self.can_be_added(biscuit, index, print_enabled=print_enabled):
            self.biscuits_and_positions.append((biscuit, index))
            self.biscuits_and_positions = sorted(self.biscuits_and_positions, key=lambda x: x[1])

            #update score 
            self.score += biscuit.value
        else:
            if print_enabled:
                print("Biscuit can't be added")
        if print_enabled:
            print("\n")
           
    def remove_biscuit(self, biscuit, index, print_enabled=False):
        if print_enabled:
            print(f"Trying to remove (biscuit_{biscuit.id}, {index}):")
        if (biscuit,index) in self.biscuits_and_positions:
            self.biscuits_and_positions.remove((biscuit,index))
            
            #update score 
            self.score -= biscuit.value
            if print_enabled:
                print("Biscuit removed")
        else:
            if print_enabled:
                print("Biscuit not in roll")
        if print_enabled:
            print("\n")   
    
    def move_biscuit(self, biscuit, index, new_index, print_enabled=False):
        self.remove_biscuit(biscuit, index, print_enabled=print_enabled)
        if not self.can_be_added(biscuit, new_index, print_enabled=print_enabled):
            self.add_biscuit(biscuit, index, print_enabled=print_enabled)
            if print_enabled:
                print("Biscuit can't be moved")
        else:
            self.add_biscuit(biscuit, new_index, print_enabled=print_enabled)
            if print_enabled:
                print("Biscuit moved")
    
    def display_roll(self):
        
        print("Roll :")
        for biscuit in self.biscuits_and_positions:
            print(f"(biscuit_{biscuit[0].id}, {biscuit[1]})", end=", ")
        print("\n")
        roll = "-"*500
        
        for biscuit in self.biscuits_and_positions:
            list1 = list(roll)
            start_index = biscuit[1]
            end_index = biscuit[1]+biscuit[0].length          
            list1[start_index: end_index] = str(biscuit[0].id)*biscuit[0].length
            roll = "".join(list1)
   
        #le final print
        for i in range(0, len(roll), 100):
            print(roll[i:i+100], end="\n")
        print("_"*100+"\n", end="\n")
            
def display_roll_defects():
        defects_count = defects.groupby(['x', 'class']).size().unstack(fill_value=0)
        print(defects_count)
            
    
roll = Roll(biscuits_and_positions=[(biscuit_0,2)], length=500)
print(roll.defects, roll.score)

{'0': 'a', '1': 'a', '2': 'a', '3': 'b', '8': 'c', '9': 'c', '10': 'c', '12': 'a', '14': 'a', '16': 'a', '19': 'b', '21': 'b', '22': 'b', '23': 'a', '25': 'a', '26': 'a', '29': 'a', '30': 'b', '32': 'a', '33': 'b', '34': 'b', '35': 'b', '36': 'a', '37': 'b', '38': 'a', '39': 'c', '40': 'c', '41': 'c', '44': 'b', '46': 'a', '47': 'c', '48': 'b', '49': 'c', '50': 'c', '51': 'c', '53': 'a', '54': 'b', '55': 'a', '57': 'c', '58': 'a', '59': 'b', '64': 'c', '65': 'c', '68': 'b', '69': 'a', '72': 'c', '73': 'a', '74': 'c', '76': 'a', '77': 'a', '82': 'c', '83': 'a', '84': 'c', '85': 'a', '89': 'c', '91': 'c', '92': 'a', '93': 'a', '94': 'c', '95': 'c', '96': 'b', '97': 'c', '98': 'c', '102': 'c', '103': 'a', '107': 'c', '108': 'a', '110': 'c', '112': 'a', '113': 'c', '115': 'c', '116': 'a', '117': 'a', '118': 'a', '119': 'a', '120': 'c', '121': 'c', '122': 'b', '123': 'b', '124': 'a', '125': 'a', '127': 'b', '128': 'c', '129': 'c', '136': 'c', '137': 'c', '139': 'c', '140': 'a', '141': 'c', 

In [139]:
roll.add_biscuit(biscuit_1, 5, print_enabled=True)
roll.remove_biscuit(biscuit_1, 5, print_enabled=True)
roll.remove_biscuit(biscuit_0, 2, print_enabled=True)
roll.add_biscuit(biscuit_3, 495, print_enabled=True)
roll.add_biscuit(biscuit_1, 10, print_enabled=True)

Trying to add (biscuit_1, 5):
Biscuit overlap
Biscuit can't be added


Trying to remove (biscuit_1, 5):
Biscuit removed


Trying to remove (biscuit_0, 2):
Biscuit not in roll


Trying to add (biscuit_3, 495):
Biscuit overlap
Biscuit can't be added


Trying to add (biscuit_1, 10):
Biscuit can be added




In [141]:
roll.move_biscuit(biscuit_3, 495, 3, print_enabled=True)

Trying to remove (biscuit_3, 495):
Biscuit removed


Biscuit can be added
Trying to add (biscuit_3, 3):
Biscuit can be added


Biscuit moved


In [142]:
roll.display_roll()
display_defects()

Roll :
(biscuit_3, 3), (biscuit_1, 10), 

---33333--11111111----------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
____________________________________________________________________________________________________

Class A
2111--------1-1-1-----12-11--1--1---1-1-----1-1-1--1-1-1--11---------1---1--12-----1-2------12-1----
---1---11---1---1221----13--------------1--21---------1-1-11---3--1--1-11---------------311---------
---1-2------1-1-1-1----1---1--1-----------3---1----1-1----------------1----211-12---1---11----1-----
-11-1--------12--------------11-1-1-----

### Genetic algorithm implementation

In [145]:
#ici c'est en fait greedy search avec remplissage à gauche

population_size, biscuits = 1, [biscuit_0, biscuit_1, biscuit_2, biscuit_3]
population = []
biscuits.sort(key=lambda x: x.value/x.length, reverse=True)

for _ in range(population_size):
    roll = Roll(biscuits_and_positions=[], length=500)
    
    for biscuit in biscuits:
        for i in range(roll.length):
            if roll.can_be_added(biscuit, i):
                roll.add_biscuit(biscuit, i)
                
    population.append(roll)
    print("Score :", roll.score)
    population[_].display_roll()

Score : 705
Roll :
(biscuit_3, 2), (biscuit_2, 7), (biscuit_3, 10), (biscuit_3, 15), (biscuit_0, 20), (biscuit_3, 24), (biscuit_3, 29), (biscuit_2, 34), (biscuit_0, 36), (biscuit_3, 41), (biscuit_0, 46), (biscuit_3, 52), (biscuit_3, 58), (biscuit_3, 63), (biscuit_3, 68), (biscuit_0, 73), (biscuit_3, 77), (biscuit_3, 84), (biscuit_0, 89), (biscuit_0, 93), (biscuit_3, 99), (biscuit_3, 104), (biscuit_0, 109), (biscuit_0, 114), (biscuit_0, 118), (biscuit_0, 122), (biscuit_3, 126), (biscuit_3, 131), (biscuit_0, 136), (biscuit_0, 140), (biscuit_3, 144), (biscuit_3, 149), (biscuit_2, 154), (biscuit_3, 158), (biscuit_2, 164), (biscuit_3, 167), (biscuit_3, 172), (biscuit_3, 179), (biscuit_2, 185), (biscuit_3, 189), (biscuit_3, 195), (biscuit_3, 200), (biscuit_3, 205), (biscuit_3, 210), (biscuit_3, 215), (biscuit_3, 220), (biscuit_3, 225), (biscuit_3, 230), (biscuit_3, 235), (biscuit_0, 242), (biscuit_3, 247), (biscuit_3, 252), (biscuit_3, 257), (biscuit_3, 262), (biscuit_3, 267), (biscuit_0, 27

In [160]:
def initialize_population(population_size, biscuits):

    time_start = time.time()
    population = []
    biscuits.sort(key=lambda x: (x.value/x.length, x.length), reverse=True)

    for _ in range(population_size):
        roll = Roll(biscuits_and_positions=[], length=500)
        
        for biscuit in biscuits:
            fails = 0
            index = random.randint(0, roll.length-1)
            while fails < 250 : #arbitrary number (the more you increase the number the better the score but it scales poorly in time)
                if roll.can_be_added(biscuit, index):
                    roll.add_biscuit(biscuit, index)
                else:
                    fails += 1
                index = random.randint(0, roll.length-1)
                    
        population.append(roll)
        # print("Score :", roll.score)
        # population[_].display_roll()
    time_end = time.time()
    print("Average time per roll :", (time_end-time_start)/population_size)
    return population

population = initialize_population(10, [biscuit_0, biscuit_1, biscuit_2, biscuit_3])

Average time per roll : 2.369012784957886


In [205]:
def fitness_population(population):
    return [roll.score for roll in population]

fitness = fitness_population(population)
fitness

[554, 578, 619, 594, 602, 620, 556, 582, 579, 568]

In [206]:
def select_elitismRoulette(population, fitness, elitism_rate=0.1):
    population_with_fitness = list(zip(population, fitness))
    population_with_fitness.sort(key=lambda x: x[1], reverse=True)
    elites = [x[0] for x in population_with_fitness[:int(len(population_with_fitness)*elitism_rate)]]

    weights = [x[1] for x in population_with_fitness]
    selected = random.choices(population_with_fitness, weights=weights, k=len(population_with_fitness)-len(elites))
    return elites + [x[0] for x in selected]

def select_elitismRank(population, fitness, elitism_rate=0.1):
    population_with_fitness = list(zip(population, fitness))
    population_with_fitness.sort(key=lambda x: x[1], reverse=True)
    elites = [x[0] for x in population_with_fitness[:int(len(population_with_fitness)*elitism_rate)]]

    weights = [x for x in range(len(population_with_fitness),0,-1)]
    selected = random.choices(population_with_fitness, weights=weights, k=len(population_with_fitness)-len(elites))
    return elites + [x[0] for x in selected]

def select_tournament(population, fitness, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

#tests
selected1 = select_elitismRoulette(population, fitness)
print(fitness_population(selected1))

selected2 = select_elitismRank(population, fitness)
print(fitness_population(selected2))

selected3 = select_tournament(population, fitness)
print(fitness_population(selected3))

[620, 582, 579, 620, 568, 579, 556, 619, 579, 620]
[620, 594, 620, 619, 620, 620, 594, 619, 620, 619]
[602, 620, 619, 620, 620, 594, 602, 602, 620, 620]
